In [123]:
from scipy.stats import pearsonr
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import WhitespaceTokenizer
from nltk.metrics import jaccard_distance
import csv
from nltk import ne_chunk,pos_tag,word_tokenize
from nltk.chunk import tree2conllstr
from nltk.tree import Tree
pd.set_option('display.max_columns',100)

In [129]:
trial_path = 'data/trial/STS.input.txt'
trial_gs_path = 'data/trial/STS.gs.txt'
trial_df = pd.read_csv(trial_path, sep='\t', lineterminator='\n', names=['sentence0','sentence1'], header=None, quoting=csv.QUOTE_NONE)
trial_gs = pd.read_csv(trial_gs_path, sep='\t', lineterminator='\n', names=['labels'], header=None, quoting=csv.QUOTE_NONE)

def na_parser(tree):
    parsed_array = []
    for chunk in tree: 
        if type(chunk) == Tree:
            word = ' '.join(leaf[0] for leaf in chunk)
            parsed_array.append(word.lower())
        else: 
            parsed_array.append(chunk[0].lower())
    return parsed_array

def preprocessing(data):
    # To see the effect of the desambiguation we do a preprocess only with it. Generaly we will use a more complete 
    # preprocessing function. 
    data = data.fillna('')
    first = lambda x: [a[0] for a in x]
    for column in data.columns:
        # words to lower
        #data[column] = data[column].str.lower()
        # desambiguate 
        data[column] = data[column].str.replace('\W', ' ')
        data[column] = data[column].apply(lambda x : pos_tag(word_tokenize(x)))
        data[column] = data[column].apply(lambda x : na_parser(ne_chunk(x,True)))
    return data

trial_df = preprocessing(trial_df)
trial_df

,sentence0,sentence1
id1,"[the, bird, is, bathing, in, the, sink]","[birdie, is, washing, itself, in, the, water, ..."
id2,"[in, may, 2010, the, troops, attempted, to, in...","[the, us, army, invaded, kabul, on, may, 7th, ..."
id3,"[john, said, he, is, considered, a, witness, b...","[he, is, not, a, suspect, anymore, john, said]"
id4,"[they, flew, out, of, the, nest, in, groups]","[they, flew, into, the, nest, together]"
id5,"[the, woman, is, playing, the, violin]","[the, young, lady, enjoys, listening, to, the,..."
id6,"[john, went, horse, back, riding, at, dawn, wi...","[sunrise, at, dawn, is, a, magnificent, view, ..."


In [125]:
def lexical_simmilarity(df):
    guess = pd.DataFrame()
    for i in df.index:
        guess.loc[i,'labels'] = jaccard_distance(set(df.loc[i,'sentence0']), set(df.loc[i,'sentence1']))
    return guess

In [126]:
guess_lex = lexical_simmilarity(trial_df)
guess_lex.head()

,labels
id1,0.727273
id2,0.750000
id3,0.363636
id4,0.600000
id5,0.909091


In [127]:
print(pearsonr(trial_gs['labels'], guess_lex['labels'])[0])

0.40498149435177305


The results are even worse than the first ones and i don't know why. 